# 1. 스타벅스 매장 정보 찾기

# For MAC

## 1) 매장 정보 찾기

In [ ]:
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 웹 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'https://www.starbucks.co.kr/store/store_map.do'
search = '문정역'

# 웹 페이지 열기
driver.get(url)
time.sleep(3)

# 검색창에 검색어 입력
search_box = driver.find_element(By.ID, 'quickSearchText')
search_box.send_keys(search)
time.sleep(1)

# 검색 버튼 클릭
search_button = driver.find_element(By.CLASS_NAME, 'quickSearchBtn')
search_button.click()

# 결과 대기 (로딩을 기다리기 위함)
time.sleep(5)

# 검색 결과 가져오기
stores = driver.find_elements(By.CSS_SELECTOR, '.quickSearchResultBox > li.quickResultLstCon')

# 결과값 파싱
store_list = []
for store in stores:
    try:
        # 매장 이름 찾기
        name = store.find_element(By.TAG_NAME, 'strong').text.strip()
        
        # 매장 주소 및 전화번호 찾기
        details = store.find_element(By.CLASS_NAME, 'result_details').text.split('\n')
        address = details[0].strip() if len(details) > 0 else ''
        phone = details[1].strip() if len(details) > 1 else ''
        
        store_data = {
            '검색어': search,
            '매장 이름': name,
            '매장 주소': address,
            '매장 번호': phone
        }
        store_list.append(store_data)
    except Exception as e:
        print(f"Error parsing store: {e}")

# JSON 형식으로 변환
result_json = json.dumps(store_list, ensure_ascii=False, indent=4)
print(result_json)

# 웹 드라이버 종료
driver.quit()


## 2) 매장 정보 데이터 프레임으로 변환

In [ ]:
import pandas as pd

# df = pd.DataFrame
df = pd.DataFrame(store_list).set_index('검색어')
df

# For Windows

## 1) 매장 정보 찾기

In [ ]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


# 웹 드라이버 실행
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do'
search = '문정역'

# 웹 페이지 열기
driver.get(url)
time.sleep(3)

# 검색창에 검색어 입력
search_box = driver.find_element(By.ID, 'quickSearchText')
search_box.send_keys(search)
time.sleep(1)

# 검색 버튼 클릭
search_button = driver.find_element(By.CLASS_NAME, 'quickSearchBtn')
search_button.click()

# 결과 대기 (로딩을 기다리기 위함)
time.sleep(5)

# 검색 결과 가져오기
stores = driver.find_elements(By.CSS_SELECTOR, '.quickSearchResultBox > li.quickResultLstCon')

# 결과값 파싱
store_list = []
for store in stores:
    try:
        # 매장 이름 찾기
        name = store.find_element(By.TAG_NAME, 'strong').text.strip()
        
        # 매장 주소 및 전화번호 찾기
        details = store.find_element(By.CLASS_NAME, 'result_details').text.split('\n')
        address = details[0].strip() if len(details) > 0 else ''
        phone = details[1].strip() if len(details) > 1 else ''
        
        store_data = {
            '검색어': search,
            '매장 이름': name,
            '매장 주소': address,
            '매장 번호': phone
        }
        store_list.append(store_data)
    except Exception as e:
        print(f"Error parsing store: {e}")

# JSON 형식으로 변환
result_json = json.dumps(store_list, ensure_ascii=False, indent=4)
print(result_json)

# 웹 드라이버 종료
driver.quit()


## 2) 매장 정보 데이터 프레임으로 변환

In [ ]:
import pandas as pd

# df = pd.DataFrame
df = pd.DataFrame(store_list).set_index('검색어')
df

# 2. 카카오맵 정보 CSV 파일로 저장

# For MAC

## 1) 카카오 맵으로 정보 불러오기 (전체 페이지)

In [ ]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

keyword = "서울 종로구 어학원"
kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"

driver.get(kakao_map_search_url)

ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list1 = []  # 결과물이 저장되는 리스트

while True:
    try:
        # 업체명, 주소, 전화번호 추출
        title = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.head_item.clickArea > strong.tit_name > a.link_name').text
        address = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.addr > p[data-id="address"]').text
        phone = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.contact.clickArea > span.phone').text
        list1.append([title, address, phone])
        ind += 1

    except NoSuchElementException:
        # 더보기 버튼 찾기
        try:
            if driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more').is_displayed():
                actions = ActionChains(driver)
                more_button = driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more')
                actions.move_to_element(more_button).click().perform()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                continue
        except (NoSuchElementException, ElementClickInterceptedException):
            pass

        # 다음 페이지로 이동
        try:
            if no >= 5:
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.next').click()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                continue
        except NoSuchElementException:
            pass

        # 5페이지 단위마다 다음 페이지 버튼 누르기
        try:
            if driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no+1}').is_displayed():
                no += 1
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no}').click()
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                continue
        except NoSuchElementException:
            break

# 데이터 프레임으로 변환
df = pd.DataFrame(list1, columns=['Title', 'Address', 'Phone'])

# CSV 파일로 저장 (현재 디렉토리에 저장됨)
df.to_csv('./kakao_map_results.csv', index=False, encoding='utf-8-sig')

# 브라우저 닫기
driver.quit()


## 2) 카카오 맵으로 정보 불러오기 (최초 10페이지)

In [1]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

keyword = "서울 종로구 어학원"
kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"

driver.get(kakao_map_search_url)

ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list1 = []  # 결과물이 저장되는 리스트

while True:
    try:
        # 업체명, 주소, 전화번호 추출
        title = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.head_item.clickArea > strong.tit_name > a.link_name').text
        address = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.addr > p[data-id="address"]').text
        phone = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.contact.clickArea > span.phone').text
        list1.append([title, address, phone])
        ind += 1

    except NoSuchElementException:
        # 더보기 버튼 찾기
        try:
            if driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more').is_displayed():
                actions = ActionChains(driver)
                more_button = driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more')
                actions.move_to_element(more_button).click().perform()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                if page == 10:
                    break
                continue
        except (NoSuchElementException, ElementClickInterceptedException):
            pass

        # 다음 페이지로 이동
        try:
            if no >= 5:
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.next').click()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                if page == 10:
                    break
                continue
        except NoSuchElementException:
            pass

        # 5페이지 단위마다 다음 페이지 버튼 누르기
        try:
            if driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no+1}').is_displayed():
                no += 1
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no}').click()
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                if page == 10:
                    break
                continue
        except NoSuchElementException:
            break

# 데이터 프레임으로 변환
df = pd.DataFrame(list1, columns=['Title', 'Address', 'Phone'])

# CSV 파일로 저장 (현재 디렉토리에 저장됨)
df.to_csv('./kakao_map_results.csv', index=False, encoding='utf-8-sig')

# 브라우저 닫기
driver.quit()


# For Windows

## 1) 카카오 맵으로 정보 불러오기 (전체 페이지)

In [ ]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome()

keyword = "서울 종로구 어학원"
kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"

driver.get(kakao_map_search_url)

ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list1 = []  # 결과물이 저장되는 리스트

while True:
    try:
        # 업체명, 주소, 전화번호 추출
        title = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.head_item.clickArea > strong.tit_name > a.link_name').text
        address = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.addr > p[data-id="address"]').text
        phone = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.contact.clickArea > span.phone').text
        list1.append([title, address, phone])
        ind += 1

    except NoSuchElementException:
        # 더보기 버튼 찾기
        try:
            if driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more').is_displayed():
                actions = ActionChains(driver)
                more_button = driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more')
                actions.move_to_element(more_button).click().perform()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                continue
        except (NoSuchElementException, ElementClickInterceptedException):
            pass

        # 다음 페이지로 이동
        try:
            if no >= 5:
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.next').click()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                continue
        except NoSuchElementException:
            pass

        # 5페이지 단위마다 다음 페이지 버튼 누르기
        try:
            if driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no+1}').is_displayed():
                no += 1
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no}').click()
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                continue
        except NoSuchElementException:
            break

# 데이터 프레임으로 변환
df = pd.DataFrame(list1, columns=['Title', 'Address', 'Phone'])

# CSV 파일로 저장 (현재 디렉토리에 저장됨)
df.to_csv('./kakao_map_results.csv', index=False, encoding='utf-8-sig')

# 브라우저 닫기
driver.quit()


## 2) 카카오 맵으로 정보 불러오기 (최초 10페이지)

In [ ]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome()

keyword = "서울 종로구 어학원"
kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"

driver.get(kakao_map_search_url)

ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list1 = []  # 결과물이 저장되는 리스트

while True:
    try:
        # 업체명, 주소, 전화번호 추출
        title = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.head_item.clickArea > strong.tit_name > a.link_name').text
        address = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.addr > p[data-id="address"]').text
        phone = driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.place\\.list > li:nth-child({ind}) > div.info_item > div.contact.clickArea > span.phone').text
        list1.append([title, address, phone])
        ind += 1

    except NoSuchElementException:
        # 더보기 버튼 찾기
        try:
            if driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more').is_displayed():
                actions = ActionChains(driver)
                more_button = driver.find_element(By.CSS_SELECTOR, '#info\\.search\\.place\\.more')
                actions.move_to_element(more_button).click().perform()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                if page == 10:
                    break
                continue
        except (NoSuchElementException, ElementClickInterceptedException):
            pass

        # 다음 페이지로 이동
        try:
            if no >= 5:
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.next').click()
                no = 1
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                if page == 10:
                    break
                continue
        except NoSuchElementException:
            pass

        # 5페이지 단위마다 다음 페이지 버튼 누르기
        try:
            if driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no+1}').is_displayed():
                no += 1
                driver.find_element(By.CSS_SELECTOR, f'#info\\.search\\.page\\.no{no}').click()
                ind = 1
                page += 1
                time.sleep(3)  # 페이지가 로드될 시간을 주기 위해 일시 중지
                if page == 10:
                    break
                continue
        except NoSuchElementException:
            break

# 데이터 프레임으로 변환
df = pd.DataFrame(list1, columns=['Title', 'Address', 'Phone'])

# CSV 파일로 저장 (현재 디렉토리에 저장됨)
df.to_csv('./kakao_map_results.csv', index=False, encoding='utf-8-sig')

# 브라우저 닫기
driver.quit()
